# TBX1 cohort

Variants in [TBX1](https://omim.org/entry/602054) are associated with the following diseases

- [Conotruncal anomaly face syndrome](https://omim.org/entry/217095)
- [DiGeorge syndrome](https://omim.org/entry/188400)
- [Tetralogy of Fallot](https://omim.org/entry/187500)
- [Velocardiofacial syndrome](https://omim.org/entry/192430)

We have assigned the individuals to the diagnosis that best fit the clinical description if the authors did not specifically state the disease diagnosis (for instance, [Ogata et al](https://pubmed.ncbi.nlm.nih.gov/24637876/) described the diagnosis as "22q11.2 Deletion Syndrome-Like Craniofacial Features and Hypocalcemia")

In [1]:
import pandas as pd
from IPython.display import display, HTML
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from collections import defaultdict
from pyphetools.creation import *
from pyphetools.visualization import *
from pyphetools.validation import *
import pyphetools
print(f"Using pyphetools version {pyphetools.__version__}")

Using pyphetools version 0.9.54


In [2]:
parser = HpoParser(hpo_json_file="../hp.json")
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
hpo_ontology = parser.get_ontology()
created_by="ORCID:0000-0002-0736-9199"
print(f"HPO version {hpo_version}")

HPO version 2024-02-27


In [3]:
df = pd.read_excel("input/TBX1_individuals.xlsx")
df.head(2)

,PMID,title,individual_id,comment,disease_id,disease_label,transcript,allele_1,allele_2,variant.comment,...,Hypoparathyroidism,Aplasia/Hypoplasia of the thymus,T lymphocytopenia,Global developmental delay,Sensorineural hearing impairment,Polydactyly,Syndactyly,Short stature,Graves disease,Miscellaneous
0,str,str,str,optional str,str,str,str,str,str,optional str,...,HP:0000829,HP:0010515,HP:0005403,HP:0001263,HP:0000407,HP:0010442,HP:0001159,HP:0004322,HP:0100647,Miscellaneous
1,PMID:24637876,TBX1 Mutation Identified by Exome Sequencing in a Japanese Family with 22q11.2 Deletion Syndrome-Like Craniofacial Features and Hypocalcemia,II-2,NaN,OMIM:188400,DiGeorge syndrome,NM_080647.1,c.1253del,na,p.(Tyr418PhefsTer42),...,observed,excluded,na,observed,observed,excluded,na,na,excluded,NaN


In [4]:
encoder = CaseTemplateEncoder(df=df, hpo_cr=hpo_cr, created_by=created_by)
individuals = encoder.get_individuals()

Created encoders for 31 fields


In [5]:
## TODO UPDATE TO MANE transcript NM_001379200.1

TBX1_transcript = "NM_080647.1" 
vmanager = VariantManager(df=df,
                          individual_column_name="individual_id",
                          gene_symbol="TBX1",
                          transcript=TBX1_transcript,
                          allele_1_column_name="allele_1",
                          allele_2_column_name="allele_2")

[INFO] encoding variant "c.1253del"
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_080647.1%3Ac.1253del/NM_080647.1?content-type=application%2Fjson


In [6]:
vmanager.to_summary()

,status,count,alleles
0,mapped,1,c.1253del
1,unmapped,0,


In [7]:
vmanager.add_variants_to_individuals(individuals)

In [8]:
cvalidator = CohortValidator(cohort=individuals, ontology=hpo_ontology, min_hpo=1,
                                allelic_requirement=AllelicRequirement.MONO_ALLELIC)
qc = QcVisualizer(cohort_validator=cvalidator)
display(HTML(qc.to_summary_html()))

In [9]:
individuals = cvalidator.get_error_free_individual_list()
table = IndividualTable(individuals)
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
II-2 (FEMALE; P51Y),DiGeorge syndrome (OMIM:188400),NM_080647.1:c.1253del (heterozygous),Hypertelorism (HP:0000316); Blepharophimosis (HP:0000581); Low-set ears (HP:0000369); Narrow nose (HP:0000460); Micrognathia (HP:0000347); Velopharyngeal insufficiency (HP:0000220); Hypoparathyroidism (HP:0000829); Global developmental delay (HP:0001263); Sensorineural hearing impairment (HP:0000407); excluded: Cleft palate (HP:0000175); excluded: Aplasia/Hypoplasia of the thymus (HP:0010515); excluded: Polydactyly (HP:0010442); excluded: Graves disease (HP:0100647)
III-1 (FEMALE; P26Y),DiGeorge syndrome (OMIM:188400),NM_080647.1:c.1253del (heterozygous),Hypertelorism (HP:0000316); Blepharophimosis (HP:0000581); Low-set ears (HP:0000369); Narrow nose (HP:0000460); Micrognathia (HP:0000347); Velopharyngeal insufficiency (HP:0000220); Global developmental delay (HP:0001263); excluded: Cleft palate (HP:0000175); excluded: Hypoparathyroidism (HP:0000829); excluded: Sensorineural hearing impairment (HP:0000407); excluded: Polydactyly (HP:0010442); excluded: Graves disease (HP:0100647)
III-5 (MALE; P19Y),DiGeorge syndrome (OMIM:188400),NM_080647.1:c.1253del (heterozygous),Hypertelorism (HP:0000316); Blepharophimosis (HP:0000581); Low-set ears (HP:0000369); Narrow nose (HP:0000460); Micrognathia (HP:0000347); Velopharyngeal insufficiency (HP:0000220); Hypoparathyroidism (HP:0000829); Global developmental delay (HP:0001263); Graves disease (HP:0100647); excluded: Cleft palate (HP:0000175); excluded: Sensorineural hearing impairment (HP:0000407); excluded: Polydactyly (HP:0010442)
III-6 (FEMALE; P13Y),DiGeorge syndrome (OMIM:188400),NM_080647.1:c.1253del (heterozygous),Hypertelorism (HP:0000316); Blepharophimosis (HP:0000581); Low-set ears (HP:0000369); Narrow nose (HP:0000460); Micrognathia (HP:0000347); Velopharyngeal insufficiency (HP:0000220); Global developmental delay (HP:0001263); excluded: Cleft palate (HP:0000175); excluded: Hypoparathyroidism (HP:0000829); excluded: Sensorineural hearing impairment (HP:0000407); excluded: Polydactyly (HP:0010442); excluded: Graves disease (HP:0100647)
III-7 (MALE; P10Y),DiGeorge syndrome (OMIM:188400),NM_080647.1:c.1253del (heterozygous),Hypertelorism (HP:0000316); Blepharophimosis (HP:0000581); Low-set ears (HP:0000369); Narrow nose (HP:0000460); Micrognathia (HP:0000347); Velopharyngeal insufficiency (HP:0000220); Hypoparathyroidism (HP:0000829); Global developmental delay (HP:0001263); excluded: Cleft palate (HP:0000175); excluded: Sensorineural hearing impairment (HP:0000407); excluded: Polydactyly (HP:0010442); excluded: Graves disease (HP:0100647)


In [10]:
# when we are finished, output phenopackets
# encoder.output_individuals_as_phenopackets(individual_list=individuals)
# Also, when we are finished, update HPOA annotations for HPO website (see https://monarch-initiative.github.io/pyphetools/developers/hpoa_editing/)